In [1]:
import pandas as pd
import pickle
from prophet import Prophet
import os

Importing plotly failed. Interactive plots will not work.


In [52]:
# Load and preprocess your data
BSinfo = pd.read_csv('..\..\Dataset\BSinfo.csv')
CLdata = pd.read_csv('..\..\Dataset\CLdata.csv')
ECdata = pd.read_csv('..\..\Dataset\ECdata.csv')
Submission = pd.read_csv('..\..\Dataset\SampleSubmission.csv')
# base = pd.read_csv('BSinfo.csv')
# cell = pd.read_csv('CLdata.csv')
# energy = pd.read_csv('ECdata.csv')

In [53]:

base_cell = pd.merge(CLdata, BSinfo, on=['BS', 'CellName'], how='left')
base_cell_energy = pd.merge(base_cell, ECdata, on=['Time', 'BS'], how='inner')

# base_cell = pd.merge(cell, base, on=['BS', 'CellName'], how='left')
# base_cell_energy = pd.merge(base_cell, energy, on=['Time', 'BS'], how='inner')

In [54]:
base_cell_energy['BS'] = base_cell_energy['BS'].str.replace('B_', '')
base_cell_energy['CellName'] = base_cell_energy['CellName'].str.replace('Cell', '')
base_cell_energy['RUType'] = base_cell_energy['RUType'].str.replace('Type', '')
base_cell_energy['Mode'] = base_cell_energy['Mode'].str.replace('Mode', '')
base_cell_energy['Time'] = pd.to_datetime(base_cell_energy['Time'])

Saving models

In [5]:
# Get unique base station values from the dataset
unique_base_stations = base_cell_energy['BS'].unique()

In [6]:
# Create a directory to save models
save_dir = 'saved_models_BS'
os.makedirs(save_dir, exist_ok=True)

In [7]:
data = base_cell_energy.copy()


In [8]:
# Resample the data and fill missing values
data.set_index('Time', inplace=True)
data_resampled = data.resample('H').mean()
data_resampled['Energy'].fillna(method='ffill', inplace=True)

In [9]:
# Rename columns for Prophet
data_resampled.rename(columns={'Energy': 'y'}, inplace=True)
data_resampled.reset_index(inplace=True)
data_resampled.rename(columns={'Time': 'ds'}, inplace=True)

In [10]:
# Create and fit the general Prophet model
general_model = Prophet()
general_model.fit(data_resampled)

19:54:58 - cmdstanpy - INFO - Chain [1] start processing
19:54:59 - cmdstanpy - INFO - Chain [1] done processing


In [11]:
# Save the general model to a file in the 'saved_models' folder
general_model_filename = os.path.join(save_dir, 'general_model.pkl')

with open(general_model_filename, 'wb') as f:
    pickle.dump(general_model, f)

In [12]:
# Train and save separate models for each base station
for bs in unique_base_stations:
    # Filter data for a specific base station
    bs_data = base_cell_energy[base_cell_energy['BS'] == bs]

    # Skip base stations with insufficient data
    if bs_data.shape[0] < 2:
        print(f"Skipping Base Station {bs} due to insufficient data")
        continue

    # Resample the data and fill missing values (rest of the preprocessing steps)
    bs_data.set_index('Time', inplace=True)
    bs_data_resampled = bs_data.resample('H').mean()
    bs_data_resampled['Energy'].fillna(method='ffill', inplace=True)
    bs_data_resampled.rename(columns={'Energy': 'y'}, inplace=True)
    bs_data_resampled.reset_index(inplace=True)
    bs_data_resampled.rename(columns={'Time': 'ds'}, inplace=True)

    # Fill missing values in the 'load' column with the mode
    mode_load = bs_data_resampled['load'].mode()[0]  # Get the first mode if there are multiple
    bs_data_resampled['load'].fillna(mode_load, inplace=True)

    # Fill missing values in the 'ESMode1' column with the mean
    mean_ES1 = bs_data_resampled['ESMode1'].mean()
    bs_data_resampled['ESMode1'].fillna(mean_ES1, inplace=True)

    # Fill missing values in the 'ESMode2' column with the mean
    mean_ES2 = bs_data_resampled['ESMode2'].mean()
    bs_data_resampled['ESMode2'].fillna(mean_ES2, inplace=True)

    # Fill missing values in the 'ESMode6' column with the mean
    mean_ES6 = bs_data_resampled['ESMode6'].mean()
    bs_data_resampled['ESMode6'].fillna(mean_ES6, inplace=True)

    bs_data_resampled.fillna(method='ffill', inplace=True)

    # Create and fit the Prophet model
    model = Prophet()
    model.add_regressor('load')
    model.add_regressor('ESMode1')
    model.add_regressor('ESMode2')
    model.add_regressor('ESMode6')
    model.add_regressor('Frequency')
    model.add_regressor('Bandwidth')
    model.add_regressor('Antennas')
    model.add_regressor('TXpower')
    model.fit(bs_data_resampled)

    # Save the model to a file in the 'saved_models' folder
    model_filename = os.path.join(save_dir, f'model_B_{bs}.pkl')
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

19:55:11 - cmdstanpy - INFO - Chain [1] start processing
19:55:11 - cmdstanpy - INFO - Chain [1] done processing
19:55:11 - cmdstanpy - INFO - Chain [1] start processing
19:55:11 - cmdstanpy - INFO - Chain [1] done processing
19:55:12 - cmdstanpy - INFO - Chain [1] start processing
19:55:12 - cmdstanpy - INFO - Chain [1] done processing
19:55:12 - cmdstanpy - INFO - Chain [1] start processing
19:55:12 - cmdstanpy - INFO - Chain [1] done processing
19:55:12 - cmdstanpy - INFO - Chain [1] start processing
19:55:13 - cmdstanpy - INFO - Chain [1] done processing
19:55:13 - cmdstanpy - INFO - Chain [1] start processing
19:55:13 - cmdstanpy - INFO - Chain [1] done processing
19:55:13 - cmdstanpy - INFO - Chain [1] start processing
19:55:13 - cmdstanpy - INFO - Chain [1] done processing
19:55:13 - cmdstanpy - INFO - Chain [1] start processing
19:55:13 - cmdstanpy - INFO - Chain [1] done processing
19:55:14 - cmdstanpy - INFO - Chain [1] start processing
19:55:14 - cmdstanpy - INFO - Chain [1]

Skipping Base Station 835 due to insufficient data


20:00:12 - cmdstanpy - INFO - Chain [1] done processing
20:00:12 - cmdstanpy - INFO - Chain [1] start processing
20:00:13 - cmdstanpy - INFO - Chain [1] done processing
20:00:13 - cmdstanpy - INFO - Chain [1] start processing
20:00:13 - cmdstanpy - INFO - Chain [1] done processing
20:00:13 - cmdstanpy - INFO - Chain [1] start processing
20:00:13 - cmdstanpy - INFO - Chain [1] done processing
20:00:13 - cmdstanpy - INFO - Chain [1] start processing
20:00:13 - cmdstanpy - INFO - Chain [1] done processing
20:00:14 - cmdstanpy - INFO - Chain [1] start processing
20:00:14 - cmdstanpy - INFO - Chain [1] done processing
20:00:14 - cmdstanpy - INFO - Chain [1] start processing
20:00:14 - cmdstanpy - INFO - Chain [1] done processing
20:00:14 - cmdstanpy - INFO - Chain [1] start processing
20:00:14 - cmdstanpy - INFO - Chain [1] done processing
20:00:14 - cmdstanpy - INFO - Chain [1] start processing
20:00:15 - cmdstanpy - INFO - Chain [1] done processing
20:00:15 - cmdstanpy - INFO - Chain [1] 

Skipping Base Station 854 due to insufficient data


20:00:15 - cmdstanpy - INFO - Chain [1] done processing
20:00:15 - cmdstanpy - INFO - Chain [1] start processing
20:00:15 - cmdstanpy - INFO - Chain [1] done processing
20:00:15 - cmdstanpy - INFO - Chain [1] start processing
20:00:15 - cmdstanpy - INFO - Chain [1] done processing
20:00:16 - cmdstanpy - INFO - Chain [1] start processing
20:00:16 - cmdstanpy - INFO - Chain [1] done processing
20:00:16 - cmdstanpy - INFO - Chain [1] start processing
20:00:16 - cmdstanpy - INFO - Chain [1] done processing
20:00:16 - cmdstanpy - INFO - Chain [1] start processing
20:00:16 - cmdstanpy - INFO - Chain [1] done processing
20:00:17 - cmdstanpy - INFO - Chain [1] start processing
20:00:17 - cmdstanpy - INFO - Chain [1] done processing
20:00:17 - cmdstanpy - INFO - Chain [1] start processing
20:00:17 - cmdstanpy - INFO - Chain [1] done processing
20:00:17 - cmdstanpy - INFO - Chain [1] start processing
20:00:17 - cmdstanpy - INFO - Chain [1] done processing
20:00:17 - cmdstanpy - INFO - Chain [1] 

Predicting

In [55]:
Submission['Time'] = pd.to_datetime(Submission['Time'], format='%Y-%d-%m %H:%M:%S')
Submission

,Time,BS,Energy,w
0,2023-01-01 06:00:00,B_0,NaN,1
1,2023-01-01 11:00:00,B_0,NaN,1
2,2023-01-01 12:00:00,B_0,NaN,1
3,2023-01-01 13:00:00,B_0,NaN,1
4,2023-01-01 23:00:00,B_0,NaN,1
...,...,...,...,...
26134,2023-02-01 19:00:00,B_1019,NaN,5
26135,2023-02-01 20:00:00,B_1019,NaN,5
26136,2023-02-01 21:00:00,B_1019,NaN,5
26137,2023-02-01 22:00:00,B_1019,NaN,5


In [56]:
base_cell['Time'] = pd.to_datetime(base_cell['Time'], format='%d/%m/%Y %H:%M')
base_cell


,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,2023-01-01 01:00:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.00,20,4,6.875934
1,2023-01-01 02:00:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.00,20,4,6.875934
2,2023-01-01 03:00:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.00,20,4,6.875934
3,2023-01-01 04:00:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.00,20,4,6.875934
4,2023-01-01 05:00:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.00,20,4,6.875934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125570,2023-02-01 19:00:00,B_745,Cell3,0.071000,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,426.98,2,2,6.877429
125571,2023-02-01 20:00:00,B_745,Cell3,0.064962,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,426.98,2,2,6.877429
125572,2023-02-01 21:00:00,B_745,Cell3,0.081865,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,426.98,2,2,6.877429
125573,2023-02-01 22:00:00,B_745,Cell3,0.097615,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,426.98,2,2,6.877429


In [57]:
cell_submit = pd.merge(Submission, base_cell, on=['Time', 'BS'], how='inner')
cell_submit['Energy'] = 0
cell_submit = cell_submit.groupby(['BS', 'Time']).apply(lambda x: x.mode().iloc[0]).reset_index(drop=True)
cell_submit= cell_submit[['Time', 'BS', 'Energy', 'load','ESMode1','ESMode2','ESMode6','Frequency', 'Bandwidth','Antennas','TXpower']]
cell_submit['Time'] = pd.to_datetime(cell_submit['Time'])

unique_base_stations_submit = cell_submit['BS'].unique()
print(unique_base_stations_submit)


['B_0' 'B_1' 'B_10' ... 'B_997' 'B_998' 'B_999']


In [58]:
# Load the general model
save_dir = 'saved_models_BS'
general_model_filename = os.path.join(save_dir, 'general_model.pkl')
with open(general_model_filename, 'rb') as f:
    general_model = pickle.load(f)

In [59]:
for bs in unique_base_stations_submit:
    # Load the corresponding Prophet model if available, otherwise use the general model
    model_filename = os.path.join(save_dir, f'model_{bs}.pkl')
    
    if os.path.exists(model_filename):
        with open(model_filename, "rb") as f:
            loaded_model = pickle.load(f)
            print('I used the specific model')
    else:
        print(bs)
        print('I used the general model')
        print(f'model_{bs}.pkl')
        # Use the general model if the specific model is not available
        loaded_model = general_model
    
    # Filter new data for the specific base station
    bs_data_submit = cell_submit[cell_submit['BS'] == bs]

    # Create the 'future' DataFrame for prediction
    future = pd.DataFrame({'ds': bs_data_submit['Time']})
    
    # Add columns to the 'future' DataFrame
    future['load'] = bs_data_submit['load']
    future['ESMode1'] = bs_data_submit['ESMode1']
    future['ESMode2'] = bs_data_submit['ESMode2']
    future['ESMode6'] = bs_data_submit['ESMode6']
    future['Frequency'] = bs_data_submit['Frequency']
    future['Bandwidth'] = bs_data_submit['Bandwidth']
    future['Antennas'] = bs_data_submit['Antennas']
    future['TXpower'] = bs_data_submit['TXpower']

    # Use the loaded model to predict values for the new data
    forecast = loaded_model.predict(future)

    # Update the 'Energy' column in the submit DataFrame with predictions
    Submission.loc[Submission['BS'] == bs, 'Energy'] = forecast['yhat'].values

I used the specific model
I used the specific model
I used the specific model
I used the specific model
B_1000
I used the general model
model_B_1000.pkl
B_1001
I used the general model
model_B_1001.pkl
B_1002
I used the general model
model_B_1002.pkl
I used the specific model
I used the specific model
I used the specific model
I used the specific model
I used the specific model
B_1008
I used the general model
model_B_1008.pkl
B_1009
I used the general model
model_B_1009.pkl
I used the specific model
B_1010
I used the general model
model_B_1010.pkl
B_1011
I used the general model
model_B_1011.pkl
B_1012
I used the general model
model_B_1012.pkl
B_1013
I used the general model
model_B_1013.pkl
B_1014
I used the general model
model_B_1014.pkl
B_1015
I used the general model
model_B_1015.pkl
B_1016
I used the general model
model_B_1016.pkl
B_1017
I used the general model
model_B_1017.pkl
I used the specific model
B_1019
I used the general model
model_B_1019.pkl
I used the specific model
I 

In [60]:
Submission['Time'] = pd.to_datetime(Submission['Time'])

# Create the 'ID' column by joining 'Time' and 'BS' columns
Submission['ID'] = Submission['Time'].apply(lambda x: x.strftime('%Y-%d-%m %H:%M:%S')).astype(str) + '_' + Submission['BS']

# Remove unnecessary columns and reorder columns
Submission = Submission[['ID', 'Energy']]

# Save the submission file
Submission.to_csv('prophet_submissions\SampleSubmission_01.csv', index=False)

print(Submission.shape)

(26139, 2)


In [61]:
Submission

,ID,Energy
0,2023-01-01 06:00:00_B_0,62.178099
1,2023-01-01 11:00:00_B_0,74.907080
2,2023-01-01 12:00:00_B_0,73.635906
3,2023-01-01 13:00:00_B_0,72.805869
4,2023-01-01 23:00:00_B_0,79.665812
...,...,...
26134,2023-01-02 19:00:00_B_1019,33.714499
26135,2023-01-02 20:00:00_B_1019,33.382161
26136,2023-01-02 21:00:00_B_1019,32.976100
26137,2023-01-02 22:00:00_B_1019,32.445420


New Model

In [126]:
# Load and preprocess your data
base = pd.read_csv('..\..\Dataset\BSinfo.csv')
cell = pd.read_csv('..\..\Dataset\CLdata.csv')
energy = pd.read_csv('..\..\Dataset\ECdata.csv')

cell.drop(['CellName'], axis=1, inplace=True)
CLdata_grouped = cell.groupby(['BS', 'Time']).sum().reset_index()
base_cell_energy = pd.merge(CLdata_grouped, energy, on=('Time','BS'), how='inner')

base_cell_energy['BS'] = base_cell_energy['BS'].str.replace('B_', '')
base_cell_energy['Time'] = pd.to_datetime(base_cell_energy['Time'], format='%m/%d/%Y %H:%M')

# Get unique base station values from the dataset
unique_base_stations = base_cell_energy['BS'].unique()

# Create a directory to save models
save_dir = 'saved_models_load_ES_sum'
os.makedirs(save_dir, exist_ok=True)

data = base_cell_energy.copy()
# Resample the data and fill missing values
data.set_index('Time', inplace=True)
data.drop(['BS'], axis=1, inplace=True)
data_resampled = data.resample('H').mean()
data_resampled['Energy'].fillna(method='ffill', inplace=True)

# Rename columns for Prophet
data_resampled.rename(columns={'Energy': 'y'}, inplace=True)
data_resampled.reset_index(inplace=True)
data_resampled.rename(columns={'Time': 'ds'}, inplace=True)

# Create and fit the general Prophet model
general_model = Prophet()
general_model.add_regressor('load')
general_model.add_regressor('ESMode1')
general_model.add_regressor('ESMode2')
general_model.add_regressor('ESMode6')
general_model.fit(data_resampled)

# Save the general model to a file in the 'saved_models' folder
general_model_filename = os.path.join(save_dir, 'general_model.pkl')
with open(general_model_filename, 'wb') as f:
    pickle.dump(general_model, f)


# Train and save separate models for each base station
for bs in unique_base_stations:
    # Filter data for a specific base station
    bs_data = base_cell_energy[base_cell_energy['BS'] == bs]

    # Skip base stations with insufficient data
    if bs_data.shape[0] < 2:
        print(f"Skipping Base Station {bs} due to insufficient data")
        continue

    # Resample the data and fill missing values (rest of the preprocessing steps)
    bs_data.set_index('Time', inplace=True)
    bs_data = bs_data.drop(['BS'], axis=1)
    bs_data_resampled = bs_data.resample('H').mean()
    bs_data_resampled['Energy'].fillna(method='ffill', inplace=True)
    bs_data_resampled.rename(columns={'Energy': 'y'}, inplace=True)
    bs_data_resampled.reset_index(inplace=True)
    bs_data_resampled.rename(columns={'Time': 'ds'}, inplace=True)

    # Fill missing values in the 'load' column with the mode
    mode_load = bs_data_resampled['load'].mode()[0]  # Get the first mode if there are multiple
    # bs_data_resampled['load'].fillna(mode_load, inplace=True)
    bs_data_resampled['load'].fillna(method='ffill', inplace=True)
    # change this to ffill

    # Fill missing values in the 'ESMode1' column with the mean
    mean_ES1 = bs_data_resampled['ESMode1'].mean()
    bs_data_resampled['ESMode1'].fillna(mean_ES1, inplace=True)

    # Fill missing values in the 'ESMode2' column with the mean
    mean_ES2 = bs_data_resampled['ESMode2'].mean()
    bs_data_resampled['ESMode2'].fillna(mean_ES2, inplace=True)

    # Fill missing values in the 'ESMode6' column with the mean
    mean_ES6 = bs_data_resampled['ESMode6'].mean()
    bs_data_resampled['ESMode6'].fillna(mean_ES6, inplace=True)

    # Create and fit the Prophet model
    model = Prophet()
    model.add_regressor('load')
    model.add_regressor('ESMode1')
    model.add_regressor('ESMode2')
    model.add_regressor('ESMode6')
    model.fit(bs_data_resampled)

    # Save the model to a file in the 'saved_models' folder
    model_filename = os.path.join(save_dir, f'model_B_{bs}.pkl')
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)


01:26:27 - cmdstanpy - INFO - Chain [1] start processing
01:26:27 - cmdstanpy - INFO - Chain [1] done processing
01:26:27 - cmdstanpy - INFO - Chain [1] start processing
01:26:27 - cmdstanpy - INFO - Chain [1] done processing
01:26:28 - cmdstanpy - INFO - Chain [1] start processing
01:26:28 - cmdstanpy - INFO - Chain [1] done processing
01:26:28 - cmdstanpy - INFO - Chain [1] start processing
01:26:28 - cmdstanpy - INFO - Chain [1] done processing
01:26:28 - cmdstanpy - INFO - Chain [1] start processing
01:26:28 - cmdstanpy - INFO - Chain [1] done processing
01:26:28 - cmdstanpy - INFO - Chain [1] start processing
01:26:28 - cmdstanpy - INFO - Chain [1] done processing
01:26:28 - cmdstanpy - INFO - Chain [1] start processing
01:26:28 - cmdstanpy - INFO - Chain [1] done processing
01:26:28 - cmdstanpy - INFO - Chain [1] start processing
01:26:29 - cmdstanpy - INFO - Chain [1] done processing
01:26:29 - cmdstanpy - INFO - Chain [1] start processing
01:26:29 - cmdstanpy - INFO - Chain [1]

Skipping Base Station 835 due to insufficient data


01:28:44 - cmdstanpy - INFO - Chain [1] done processing
01:28:44 - cmdstanpy - INFO - Chain [1] start processing
01:28:44 - cmdstanpy - INFO - Chain [1] done processing
01:28:45 - cmdstanpy - INFO - Chain [1] start processing
01:28:45 - cmdstanpy - INFO - Chain [1] done processing
01:28:45 - cmdstanpy - INFO - Chain [1] start processing
01:28:45 - cmdstanpy - INFO - Chain [1] done processing
01:28:45 - cmdstanpy - INFO - Chain [1] start processing
01:28:45 - cmdstanpy - INFO - Chain [1] done processing
01:28:45 - cmdstanpy - INFO - Chain [1] start processing
01:28:46 - cmdstanpy - INFO - Chain [1] done processing
01:28:46 - cmdstanpy - INFO - Chain [1] start processing
01:28:46 - cmdstanpy - INFO - Chain [1] done processing
01:28:46 - cmdstanpy - INFO - Chain [1] start processing
01:28:46 - cmdstanpy - INFO - Chain [1] done processing
01:28:46 - cmdstanpy - INFO - Chain [1] start processing
01:28:46 - cmdstanpy - INFO - Chain [1] done processing
01:28:46 - cmdstanpy - INFO - Chain [1] 

Skipping Base Station 854 due to insufficient data


01:28:47 - cmdstanpy - INFO - Chain [1] done processing
01:28:47 - cmdstanpy - INFO - Chain [1] start processing
01:28:47 - cmdstanpy - INFO - Chain [1] done processing
01:28:48 - cmdstanpy - INFO - Chain [1] start processing
01:28:48 - cmdstanpy - INFO - Chain [1] done processing
01:28:48 - cmdstanpy - INFO - Chain [1] start processing
01:28:48 - cmdstanpy - INFO - Chain [1] done processing
01:28:48 - cmdstanpy - INFO - Chain [1] start processing
01:28:48 - cmdstanpy - INFO - Chain [1] done processing
01:28:48 - cmdstanpy - INFO - Chain [1] start processing
01:28:48 - cmdstanpy - INFO - Chain [1] done processing
01:28:49 - cmdstanpy - INFO - Chain [1] start processing
01:28:49 - cmdstanpy - INFO - Chain [1] done processing
01:28:49 - cmdstanpy - INFO - Chain [1] start processing
01:28:49 - cmdstanpy - INFO - Chain [1] done processing
01:28:49 - cmdstanpy - INFO - Chain [1] start processing
01:28:49 - cmdstanpy - INFO - Chain [1] done processing
01:28:49 - cmdstanpy - INFO - Chain [1] 

In [128]:
# Load the new dataset
base = pd.read_csv('..\..\Dataset\BSinfo.csv')
cell = pd.read_csv('..\..\Dataset\CLdata.csv')
energy = pd.read_csv('..\..\Dataset\ECdata.csv')
submit = pd.read_csv('..\..\Dataset\SampleSubmission.csv')

CLdata_grouped = cell.groupby(['BS', 'Time']).sum().reset_index()

submit['Time'] = pd.to_datetime(submit['Time'], format='%Y-%m-%d %H:%M:%S')
CLdata_grouped['Time'] = pd.to_datetime(CLdata_grouped['Time'], format='%m/%d/%Y %H:%M')

cell_submit = pd.merge(CLdata_grouped, submit, on=('Time','BS'), how='inner')

cell_submit['Energy'] = 0
cell_submit= cell_submit[['Time', 'BS', 'Energy', 'load','ESMode1','ESMode2','ESMode6']]
cell_submit['Time'] = pd.to_datetime(cell_submit['Time'])
unique_base_stations_submit = cell_submit['BS'].unique()
print(unique_base_stations_submit)

# Load the general model
with open(general_model_filename, 'rb') as f:
    general_model = pickle.load(f)

for bs in unique_base_stations_submit:
    # Load the corresponding Prophet model if available, otherwise use the general model
    model_filename = os.path.join(save_dir, f'model_{bs}.pkl')
    
    if os.path.exists(model_filename):
        with open(model_filename, "rb") as f:
            loaded_model = pickle.load(f)
            print('Specific model used.')
    else:
        print(bs)
        print(f'model_{bs}.pkl')
        # Use the general model if the specific model is not available
        loaded_model = general_model
        print('General model used.')
    
    # Filter new data for the specific base station
    bs_data_submit = cell_submit[cell_submit['BS'] == bs]

    # Create the 'future' DataFrame for prediction
    future = pd.DataFrame({'ds': bs_data_submit['Time']})
    
    # Add columns to the 'future' DataFrame
    future['load'] = bs_data_submit['load']
    future['ESMode1'] = bs_data_submit['ESMode1']
    future['ESMode2'] = bs_data_submit['ESMode2']
    future['ESMode6'] = bs_data_submit['ESMode6']

    # Use the loaded model to predict values for the new data
    forecast = loaded_model.predict(future)

    # Update the 'Energy' column in the submit DataFrame with predictions
    submit.loc[submit['BS'] == bs, 'Energy'] = forecast['yhat'].values

submit['Time'] = pd.to_datetime(submit['Time'])

# Create the 'ID' column by joining 'Time' and 'BS' columns
submit['ID'] = submit['Time'].astype(str) + '_' + submit['BS']

# Remove unnecessary columns and reorder columns
submit = submit[['ID', 'Energy']]

# Save the submission file
submit.to_csv('prophet_submissions\SampleSubmission_06.csv', index=False)
print(submit.shape)
submit

['B_0' 'B_1' 'B_10' ... 'B_997' 'B_998' 'B_999']
Specific model used.
Specific model used.
Specific model used.
Specific model used.
B_1000
model_B_1000.pkl
General model used.
B_1001
model_B_1001.pkl
General model used.
B_1002
model_B_1002.pkl
General model used.
Specific model used.
Specific model used.
Specific model used.
Specific model used.
Specific model used.
B_1008
model_B_1008.pkl
General model used.
B_1009
model_B_1009.pkl
General model used.
Specific model used.
B_1010
model_B_1010.pkl
General model used.
B_1011
model_B_1011.pkl
General model used.
B_1012
model_B_1012.pkl
General model used.
B_1013
model_B_1013.pkl
General model used.
B_1014
model_B_1014.pkl
General model used.
B_1015
model_B_1015.pkl
General model used.
B_1016
model_B_1016.pkl
General model used.
B_1017
model_B_1017.pkl
General model used.
Specific model used.
B_1019
model_B_1019.pkl
General model used.
Specific model used.
Specific model used.
Specific model used.
Specific model used.
Specific model used.

,ID,Energy
0,2023-01-01 06:00:00_B_0,61.408711
1,2023-01-01 11:00:00_B_0,70.639561
2,2023-01-01 12:00:00_B_0,68.007749
3,2023-01-01 13:00:00_B_0,68.966991
4,2023-01-01 23:00:00_B_0,78.984117
...,...,...
26134,2023-01-02 19:00:00_B_1019,23.242183
26135,2023-01-02 20:00:00_B_1019,23.053086
26136,2023-01-02 21:00:00_B_1019,22.796842
26137,2023-01-02 22:00:00_B_1019,22.390565
